<a href="https://cognitiveclass.ai"><img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork/images/IDSN-logo.png" width="400"> </a>

# Convolutional Neural Networks with Keras

Estimated time needed **30** mins


In this lab, we will learn how to use the Keras library to build convolutional neural networks. We will also use the popular MNIST dataset and we will compare our results to using a conventional neural network.


## Objectives for this Notebook    
* How to use the Keras library to build convolutional neural networks
* Convolutional neural network with one set of convolutional and pooling layers
* Convolutional neural network with two sets of convolutional and pooling layers



## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
      
1. <a href="#Import-Keras-and-Packages">Import Keras and Packages</a>   
2. <a href="#Convolutional-Neural-Network-with-One-Set-of-Convolutional-and-Pooling-Layers">Convolutional Neural Network with One Set of Convolutional and Pooling Layers</a>  
3. <a href="#Convolutional-Neural-Network-with-Two-Sets-of-Convolutional-and-Pooling-Layers">Convolutional Neural Network with Two Sets of Convolutional and Pooling Layers</a>  

</font>
</div>


### Install the necessary libraries


Let's start by installing the keras libraries and the packages that we would need to build a neural network.


In [1]:
# All Libraries required for this lab are listed below. The libraries need to be installed on Skills Network Labs. 
# If you run this notebook on a different environment, e.g. your desktop, you may want to install these.
!pip install numpy==2.0.2
!pip install pandas==2.2.2
!pip install tensorflow_cpu==2.18.0
!pip install matplotlib==3.9.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 129.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 131.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 MB 37.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 48.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 168.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 168.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.

#### Suppress the tensorflow warning messages
We use the following code to  suppress the warning messages due to use of CPU architechture for tensoflow.
You may want to **comment out** these lines if you are using the GPU architechture


In [2]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## Import Keras and Packages


In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
from keras.utils import to_categorical

When working with convolutional neural networks in particular, we will need additional packages.


In [4]:
from keras.layers import Conv2D # to add convolutional layers
from keras.layers import MaxPooling2D # to add pooling layers
from keras.layers import Flatten # to flatten data for fully connected layers

## Convolutional Neural Network with One Set of Convolutional and Pooling Layers


In [5]:
# import data
from keras.datasets import mnist

# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Let's normalize the pixel values to be between 0 and 1


In [6]:
X_train = X_train / 255 # normalize training data
X_test = X_test / 255 # normalize test data

Next, let's convert the target variable into binary categories


In [7]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = y_test.shape[1] # number of categories

Next, let's define a function that creates our model. Let's start with one set of convolutional and pooling layers.


In [8]:
def convolutional_model():
    
    # create model
    model = Sequential()
    model.add(Input(shape=(28, 28, 1)))
    model.add(Conv2D(16, (5, 5), strides=(1, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    return model

Finally, let's call the function to create the model, and then let's train it and evaluate it.


In [9]:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))

Epoch 1/10
300/300 - 35s - 118ms/step - accuracy: 0.9172 - loss: 0.2951 - val_accuracy: 0.9688 - val_loss: 0.1074
Epoch 2/10
300/300 - 41s - 137ms/step - accuracy: 0.9732 - loss: 0.0920 - val_accuracy: 0.9802 - val_loss: 0.0639
Epoch 3/10
300/300 - 41s - 136ms/step - accuracy: 0.9822 - loss: 0.0614 - val_accuracy: 0.9839 - val_loss: 0.0498
Epoch 4/10
300/300 - 34s - 112ms/step - accuracy: 0.9855 - loss: 0.0484 - val_accuracy: 0.9846 - val_loss: 0.0467
Epoch 5/10
300/300 - 34s - 112ms/step - accuracy: 0.9883 - loss: 0.0385 - val_accuracy: 0.9864 - val_loss: 0.0396
Epoch 6/10
300/300 - 34s - 112ms/step - accuracy: 0.9901 - loss: 0.0327 - val_accuracy: 0.9879 - val_loss: 0.0362
Epoch 7/10
300/300 - 33s - 111ms/step - accuracy: 0.9914 - loss: 0.0274 - val_accuracy: 0.9875 - val_loss: 0.0360
Epoch 8/10
300/300 - 33s - 111ms/step - accuracy: 0.9928 - loss: 0.0228 - val_accuracy: 0.9879 - val_loss: 0.0364
Epoch 9/10
300/300 - 33s - 109ms/step - accuracy: 0.9938 - loss: 0.0203 - val_accuracy: 

------------------------------------------


## Convolutional Neural Network with Two Sets of Convolutional and Pooling Layers


Let's redefine our convolutional model so that it has two convolutional and pooling layers instead of just one layer of each.


In [10]:
def convolutional_model():
    
    # create model
    model = Sequential()
    model.add(Input(shape=(28, 28, 1)))
    model.add(Conv2D(16, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Conv2D(8, (2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    return model

Now, let's call the function to create our new convolutional neural network, and then let's train it and evaluate it.


In [11]:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))

Epoch 1/10
300/300 - 37s - 123ms/step - accuracy: 0.8549 - loss: 0.5160 - val_accuracy: 0.9556 - val_loss: 0.1616
Epoch 2/10
300/300 - 36s - 118ms/step - accuracy: 0.9573 - loss: 0.1442 - val_accuracy: 0.9728 - val_loss: 0.0953
Epoch 3/10
300/300 - 36s - 119ms/step - accuracy: 0.9694 - loss: 0.1008 - val_accuracy: 0.9764 - val_loss: 0.0825
Epoch 4/10
300/300 - 36s - 119ms/step - accuracy: 0.9762 - loss: 0.0789 - val_accuracy: 0.9821 - val_loss: 0.0605
Epoch 5/10
300/300 - 36s - 119ms/step - accuracy: 0.9799 - loss: 0.0664 - val_accuracy: 0.9805 - val_loss: 0.0632
Epoch 6/10
300/300 - 36s - 119ms/step - accuracy: 0.9826 - loss: 0.0575 - val_accuracy: 0.9852 - val_loss: 0.0476
Epoch 7/10
300/300 - 36s - 119ms/step - accuracy: 0.9846 - loss: 0.0505 - val_accuracy: 0.9856 - val_loss: 0.0428
Epoch 8/10
300/300 - 36s - 119ms/step - accuracy: 0.9862 - loss: 0.0443 - val_accuracy: 0.9895 - val_loss: 0.0357
Epoch 9/10
300/300 - 36s - 119ms/step - accuracy: 0.9874 - loss: 0.0399 - val_accuracy: 

<h3>Practice Exercise 1</h3>


Let's see how batch size affects the time required and accuracy of the model training. 
For this, you can try to change batch_size to 1024 and check it's effect on accuracy


In [12]:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))

Epoch 1/10
59/59 - 36s - 608ms/step - accuracy: 0.6229 - loss: 1.3235 - val_accuracy: 0.8800 - val_loss: 0.4099
Epoch 2/10
59/59 - 34s - 570ms/step - accuracy: 0.9046 - loss: 0.3226 - val_accuracy: 0.9349 - val_loss: 0.2270
Epoch 3/10
59/59 - 33s - 563ms/step - accuracy: 0.9403 - loss: 0.2055 - val_accuracy: 0.9563 - val_loss: 0.1503
Epoch 4/10
59/59 - 33s - 567ms/step - accuracy: 0.9561 - loss: 0.1498 - val_accuracy: 0.9656 - val_loss: 0.1172
Epoch 5/10
59/59 - 33s - 560ms/step - accuracy: 0.9643 - loss: 0.1216 - val_accuracy: 0.9716 - val_loss: 0.0954
Epoch 6/10
59/59 - 33s - 555ms/step - accuracy: 0.9692 - loss: 0.1041 - val_accuracy: 0.9748 - val_loss: 0.0850
Epoch 7/10
59/59 - 33s - 562ms/step - accuracy: 0.9723 - loss: 0.0912 - val_accuracy: 0.9770 - val_loss: 0.0755
Epoch 8/10
59/59 - 33s - 563ms/step - accuracy: 0.9754 - loss: 0.0824 - val_accuracy: 0.9774 - val_loss: 0.0728
Epoch 9/10
59/59 - 34s - 568ms/step - accuracy: 0.9776 - loss: 0.0754 - val_accuracy: 0.9792 - val_loss:

<h3>Practice Exercise 2</h3>


Now, let's see how number of epochs  affect the time required and accuracy of the model training. 
For this, you can keep the batch_size=1024 and epochs=25 and check it's effect on accuracy


In [13]:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))

Epoch 1/25
59/59 - 35s - 600ms/step - accuracy: 0.6754 - loss: 1.2578 - val_accuracy: 0.8753 - val_loss: 0.4357
Epoch 2/25
59/59 - 33s - 566ms/step - accuracy: 0.9042 - loss: 0.3332 - val_accuracy: 0.9375 - val_loss: 0.2241
Epoch 3/25
59/59 - 33s - 564ms/step - accuracy: 0.9431 - loss: 0.1954 - val_accuracy: 0.9576 - val_loss: 0.1490
Epoch 4/25
59/59 - 41s - 695ms/step - accuracy: 0.9604 - loss: 0.1394 - val_accuracy: 0.9675 - val_loss: 0.1105
Epoch 5/25
59/59 - 33s - 564ms/step - accuracy: 0.9675 - loss: 0.1134 - val_accuracy: 0.9729 - val_loss: 0.0897
Epoch 6/25
59/59 - 33s - 564ms/step - accuracy: 0.9725 - loss: 0.0956 - val_accuracy: 0.9741 - val_loss: 0.0843
Epoch 7/25
59/59 - 33s - 560ms/step - accuracy: 0.9758 - loss: 0.0841 - val_accuracy: 0.9778 - val_loss: 0.0696
Epoch 8/25
59/59 - 33s - 565ms/step - accuracy: 0.9782 - loss: 0.0753 - val_accuracy: 0.9804 - val_loss: 0.0636
Epoch 9/25
59/59 - 33s - 563ms/step - accuracy: 0.9798 - loss: 0.0685 - val_accuracy: 0.9819 - val_loss:

### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!


<!--
## Change Log

|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
| 2024-11-20  | 3.0  | Aman  |  Updated the library versions to current |
| 2020-09-21  | 2.0  | Srishti  |  Migrated Lab to Markdown and added to course repo in GitLab |



<hr>

## <h3 align="center"> © IBM Corporation. All rights reserved. <h3/>


## <h3 align="center"> &#169; IBM Corporation. All rights reserved. <h3/>

